# Loading and disaplying the 3DMM

In [ ]:
# import packages
import pickle
import numpy as np
import trimesh
import pandas as pd
import time
from datetime import datetime
import plotly.io as pio
import os
import copy

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from Prediction_functions import *

In [ ]:
media_folder = "./datasets/TMS"

## Original model including everything

In [ ]:
# load model
models_folder = media_folder+'/3DMM/UHM_models/'
currnet_model = 'UHM' 

# UHM model with all the components fused together (i.e. ears, inner mouth, and teeth)
model_name = 'head_model_global_align'

model_file = open(models_folder + model_name + '.pkl', 'rb')
model_dict = pickle.load(model_file)
model_file.close()

In [ ]:
# turning coordinates system to be in millimeter units
scale_factor = 100 

# get model parameteres
mean_shape = scale_factor*model_dict['Mean']
mean_shape_CCS = mean_shape.reshape(-1,3)
eigen_vec = model_dict['Eigenvectors']
eigen_vec_num =  model_dict['Eigenvectors'].shape[1]
eigen_val = model_dict['EigenValues']
trilist = model_dict['Trilist']
vertices_num = model_dict['Number_of_vertices']

In [ ]:
# load modules (landmarks and masks)
modules_folder = models_folder + '/Landmarks and masks/'

modules_to_load = ['68_land_idxs'] # EEG_10_20_full_model / '49_plus_ears_land_idxs' / '68_land_idxs'

landmarks = []
landmarks_names = []
landmarks_groups = []

for currnet_module_name in modules_to_load:
    module_file = open(modules_folder + currnet_module_name + '.pkl', 'rb')
    currnet_module = pickle.load(module_file)
    module_file.close()
    if currnet_module_name=='EEG_10_20':
        currnet_module_names = list(currnet_module.keys())
        currnet_module = np.asarray(list(currnet_module.values()))
    else:
        currnet_module_names = list(map(str, 1+np.arange(len(currnet_module))))
        
    landmarks.append(currnet_module)
    landmarks_names.append(currnet_module_names)
    landmarks_groups.append(np.arange(len(currnet_module)))

# turn list of lists into one list
landmarks = [item for items in landmarks for item in items]
landmarks_names = [item for items in landmarks_names for item in items]

num_of_landmarks = len(landmarks)

## Lighter model including everything but eyes, teeth and inner mouth cavity

In [ ]:
# load model
light_models_folder = media_folder+'/3DMM/UHM_models/'
light_currnet_model = 'UHM' 

# UHM model with all the components fused together (i.e. ears, inner mouth, and teeth)
light_model_name = 'head_model_global_align_no_mouth_and_eyes'

light_model_file = open(light_models_folder + light_model_name + '.pkl', 'rb')
light_model_dict = pickle.load(light_model_file)
light_model_file.close()

In [ ]:
# turning cartesian coordinates system to be in millimeter units
light_scale_factor = 100

# get model parameteres
light_mean_shape = light_scale_factor*light_model_dict['Mean']
light_mean_shape_CCS = light_mean_shape.reshape(-1,3)
light_eigen_vec = light_model_dict['Eigenvectors']
light_eigen_vec_num =  light_model_dict['Eigenvectors'].shape[1]
light_eigen_val = light_model_dict['EigenValues']
light_trilist = light_model_dict['Trilist']
light_vertices_num = light_model_dict['Number_of_vertices']

In [ ]:
# load modules (landmarks and masks)
modules_folder = models_folder + '/Landmarks and masks/'

modules_to_load = ['EEG_10_20'] # EEG_10_20 / '49_plus_ears_land_idxs' / '68_land_idxs'

light_landmarks = []
light_landmarks_names = []
light_landmarks_groups = []

for currnet_module_name in modules_to_load:
    module_file = open(modules_folder + currnet_module_name + '.pkl', 'rb')
    currnet_module = pickle.load(module_file)
    module_file.close()
    if currnet_module_name=='EEG_10_20':
        currnet_module_names = list(currnet_module.keys())
        currnet_module = np.asarray(list(currnet_module.values()))
    else:
        currnet_module_names = list(map(str, 1+np.arange(len(currnet_module))))
        
    light_landmarks.append(currnet_module)
    light_landmarks_names.append(currnet_module_names)
    light_landmarks_groups.append(np.arange(len(currnet_module)))

# turn list of lists into one list
light_landmarks = [item for items in light_landmarks for item in items]
light_landmarks_names = [item for items in light_landmarks_names for item in items]

num_of_light_landmarks = len(light_landmarks)

## Matching landmark indices between models

In [ ]:
light_facial_landmarks = landmarks

for current_landmark_index, current_landmark_vertex in enumerate(landmarks):
    original_model_coordinates = mean_shape_CCS[current_landmark_vertex]
    new_model_vertex_diffs = np.linalg.norm(light_mean_shape_CCS-original_model_coordinates, axis=1)
    light_facial_landmarks[current_landmark_index] = np.argmin(new_model_vertex_diffs)

In [ ]:
light_nasion = 52241
light_inion = 36323

# Definitions

## Model Choosing

In [ ]:
choose_light_model = True

In [ ]:
if choose_light_model:
    landmarks = np.concatenate((light_landmarks, light_facial_landmarks, [light_nasion, light_inion]))
    landmarks_names = list(np.concatenate((light_landmarks_names, landmarks_names, ['nasion', 'inion'])))
    num_of_landmarks = len(landmarks_names)
    mean_shape = light_mean_shape
    mean_shape_CCS = light_mean_shape_CCS
    eigen_vec = light_eigen_vec
    eigen_vec_num =  light_eigen_vec_num
    eigen_val = light_eigen_val
    trilist = light_trilist
    vertices_num = light_vertices_num

## Parameters

In [ ]:
mean_shape_trimesh = trimesh.Trimesh(vertices=mean_shape_CCS, faces=trilist, process=True)
num_digits_round=4
n_jobs_num=6

In [ ]:
distinct_landmarks_names = np.array([37, 40, 43, 46, 49, 55, 31, 9])
rigid_facial_landmarks_names = np.array([37, 40, 43, 46, 28, 1, 17])

center_of_the_eyebrows = np.array([20, 25])
corners_of_the_eyebrows = np.array([18, 22, 23, 27])
corners_of_the_eyes = np.array([37, 40, 43, 46])
sides_of_the_face = np.array([1, 17])
nose_bone = np.array([28, 31])
lower_nose = np.array([32, 34, 36])
corners_of_the_mouth = np.array([49, 55])
chin = np.array([9])

facial_landmarks = np.concatenate((center_of_the_eyebrows, corners_of_the_eyebrows, corners_of_the_eyes, sides_of_the_face, nose_bone, lower_nose,
                                   corners_of_the_mouth, chin))
selected_facial_indices = np.sort(facial_landmarks+num_of_light_landmarks-1)

selected_EEG_10_20_landmark_names = light_landmarks_names
selected_EEG_10_20_indices = []
for current_index, current_landmark_name in enumerate(selected_EEG_10_20_landmark_names):
    selected_EEG_10_20_indices.append(landmarks_names.index(current_landmark_name))
selected_EEG_10_20_indices = np.asarray(selected_EEG_10_20_indices)

In [ ]:
input_landmarks = ['1', '2', '3', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
                   '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
                   '43', '44', '45', '46', '47', '48', '69', '70']
input_landmarks = [int(current_landmark) for current_landmark in input_landmarks]

In [ ]:
selected_indices = np.concatenate((selected_EEG_10_20_indices, 20+np.array(input_landmarks)))
selected_indices_names = np.take(landmarks_names, selected_indices)

In [ ]:
feature_sets = {}
feature_sets['eye corners & eyebrow corners'] = np.concatenate((corners_of_the_eyebrows, corners_of_the_eyes))
feature_sets['eye corners & eyebrow centers'] = np.concatenate((center_of_the_eyebrows, corners_of_the_eyes))
feature_sets['eye corners & eyebrow corners and center'] = np.concatenate((center_of_the_eyebrows, corners_of_the_eyebrows, corners_of_the_eyes))
feature_sets['eye corners & nose bone'] = np.concatenate((corners_of_the_eyes, nose_bone))
feature_sets['nose bone & lower nose'] = np.concatenate((nose_bone, lower_nose))
feature_sets['input_landmarks'] = np.array(input_landmarks[:-2])

for current_key in feature_sets:
    feature_sets[current_key] = feature_sets[current_key]+num_of_light_landmarks-1
    feature_sets[current_key] = list(map(str, feature_sets[current_key]))

## Functions

# Loading instances dataframe

In [ ]:
folder = media_folder+"/3DMM/Head_instances/"
filetype = ".xlsx"
fixed_filename = "dataset" 
fixed_path = folder + fixed_filename + filetype

excel_file = pd.ExcelFile(fixed_path, engine='openpyxl')

In [ ]:
# read facial features coordinates from an excel file as multiindex
fixed_landmarks_coordinates_df=pd.read_excel(fixed_path, header=[0,1], index_col=0, sheet_name=('coordinates_' + str(1)), engine='openpyxl')
fixed_geodesic_distances_df=pd.read_excel(fixed_path, index_col=0, sheet_name=('geodesic_' + str(1)), engine='openpyxl')

fixed_num_of_instances = fixed_landmarks_coordinates_df.shape[0]
fixed_landmarks_coordinates_df = fixed_landmarks_coordinates_df/1000
fixed_geodesic_distances_df = fixed_geodesic_distances_df/1000

# Landmark Predictions - MLP

In [ ]:
MLP_nn = MLP_3

In [ ]:
save_figures = False
save_arrays = True
load_arrays = False
save_model = True
regressor_name='pytorch_MLP'
regressor_models_folder = media_folder+'/3DMM/Trained_models/' + regressor_name + '/'

In [ ]:
np.random.seed(0)

test_size=0.0001

train_indices = np.sort(np.random.choice(range(fixed_landmarks_coordinates_df.shape[0]),
                                         #1, replace=False))
                                 int(fixed_landmarks_coordinates_df.shape[0]*(1-test_size)), replace=False))
test_indices = np.setdiff1d(np.arange(fixed_landmarks_coordinates_df.shape[0]), train_indices)#

## Predict coordinates using coordinates

In [ ]:
experiment_number = 1

experiment_model_filename = 'Coordinates/'+MLP_folder+'3DMM/'
experiment_model_path = regressor_models_folder + experiment_model_filename

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

if load_arrays==False:
    coordinates_coordinates_resolutions_MSE_array = np.zeros((len(selected_EEG_10_20_indices),))
    coordinates_coordinates_resolutions_std_array = np.zeros((len(selected_EEG_10_20_indices),))
    validation_losses = []
    test_losses = []
    
    feature_set_index=len(list(feature_sets.keys()))-1

    current_features = feature_sets[list(feature_sets.keys())[feature_set_index]]
    for desired_landmark_index, desired_landmark_name in enumerate((np.take(landmarks_names, selected_EEG_10_20_indices))):
        model, coordinates_coordinates_resolutions_MSE_array[desired_landmark_index], _, coordinates_coordinates_resolutions_std_array[
            desired_landmark_index], scaler, means, validation_loss, test_loss = coordinates_by_coordinates_regression(
            fixed_landmarks_coordinates_df, train_indices, test_indices, current_features, desired_landmark_index, landmarks_names, regressor_name)

        validation_losses.append(validation_loss)
        test_losses.append(test_loss)

        print(1000*np.sqrt(coordinates_coordinates_resolutions_MSE_array[desired_landmark_index]))

        if save_model:
            timestamp_string = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
            timestamp_string = timestamp_string.replace('_2022_', '_22_')

            torch.save(model.state_dict(), experiment_model_path+timestamp_string+'_'+desired_landmark_name+'_model')
            pickle.dump(scaler, open(experiment_model_path+timestamp_string+'_'+desired_landmark_name+'_scaler.pkl', 'wb'))
            documentation = [
                f"predicted_landmark_name: {desired_landmark_name}",
                f"landmark_names_being_used: {list(np.sort(np.array(list(map(int, current_features)))-20))}",
                f"number_of_training_samples: {train_indices.size}",
                f"means: {means}",
                f"model: {model}",
            ]
            with open(experiment_model_path+timestamp_string+'_'+desired_landmark_name+'_documentation.txt' , "w") as txt_file:
                txt_file.write("\n".join(documentation))

        print(f"Finished {desired_landmark_name}, {desired_landmark_index+1}/{len(selected_EEG_10_20_indices)}")

else:
    coordinates_coordinates_resolutions_MSE_array, coordinates_coordinates_resolutions_std_array = experiment_arrays_loader(media_folder, regressor_name, experiment_number)
    save_arrays = False

## Predict coordinates using coordinates and geodesic distances

In [ ]:
experiment_number = 2

experiment_model_filename = 'Coordinates_Geodesic/'+MLP_folder+'3DMM/'

experiment_model_path = regressor_models_folder + experiment_model_filename

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

if load_arrays==False:
    coordinates_coordinates_resolutions_MSE_array = np.zeros((len(selected_EEG_10_20_indices),))
    coordinates_coordinates_resolutions_std_array = np.zeros((len(selected_EEG_10_20_indices),))
    validation_losses = []
    test_losses = []
    
    feature_set_index=len(list(feature_sets.keys()))-1

    current_features = feature_sets[list(feature_sets.keys())[feature_set_index]]
    for desired_landmark_index, desired_landmark_name in enumerate((np.take(landmarks_names, selected_EEG_10_20_indices))):
        
        model, coordinates_coordinates_resolutions_MSE_array[desired_landmark_index], _, coordinates_coordinates_resolutions_std_array[
            desired_landmark_index], scaler, means, validation_loss, test_loss = coordinates_by_coordinates_and_geodesic_distance_regression(
            fixed_landmarks_coordinates_df, fixed_geodesic_distances_df, train_indices, test_indices,current_features,
            desired_landmark_index, landmarks_names, regressor_name)

        validation_losses.append(validation_loss)
        test_losses.append(test_loss)

        print(1000*np.sqrt(coordinates_coordinates_resolutions_MSE_array[desired_landmark_index]))

        if save_model:
            timestamp_string = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
            timestamp_string = timestamp_string.replace('_2022_', '_22_')

            torch.save(model.state_dict(), experiment_model_path+timestamp_string+'_'+desired_landmark_name+'_model')
            pickle.dump(scaler, open(experiment_model_path+timestamp_string+'_'+desired_landmark_name+'_scaler.pkl', 'wb'))
            documentation = [
                f"predicted_landmark_name: {desired_landmark_name}",
                f"landmark_names_being_used: {list(np.sort(np.array(list(map(int, current_features)))-20))}",
                f"number_of_training_samples: {train_indices.size}",
                f"means: {means}",
                f"model: {model}",
            ]
            with open(experiment_model_path+timestamp_string+'_'+desired_landmark_name+'_documentation.txt' , "w") as txt_file:
                txt_file.write("\n".join(documentation))

        print(f"Finished {desired_landmark_name}, {desired_landmark_index+1}/{len(selected_EEG_10_20_indices)}")

else:
    coordinates_coordinates_resolutions_MSE_array, coordinates_coordinates_resolutions_std_array = experiment_arrays_loader(media_folder, regressor_name, experiment_number)
    save_arrays = False